# Sales Data Preparation for Power BI Dashboard

This project focuses on preparing sales data from the **Global Superstore dataset** using Python.  
The data cleaning and transformation steps are performed in Google Colab, and the processed data will be exported for visualization and reporting in Power BI.

The quality of data is fundamental for accurate analysis and effective strategic decision-making based on business dashboards. We aim to produce clean, reliable, and well-structured data to support insightful business dashboards.

**Dataset**: Global Superstore (Kaggle version)  
**Author**: Paulo Castro  
**Date**: July 2025  
**Tools**: Python (pandas), Power BI

---

## 1. Loading and Inspecting the Data

In this section, we load the dataset and perform an initial inspection to understand its structure and basic characteristics.

We focus on:
- The number of rows and columns
- Data types of each column
- General completeness of the data

> **Note**: The file was imported using `encoding='latin1'` due to character encoding issues with special characters in the original CSV file.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/Global_Superstore2.csv', encoding='latin1')
df.info()

### 1.1 Initial Data Overview from `df.info()`

Upon loading the dataset and examining its structure using `df.info()`, we observe the following key characteristics:

* **Total Entries**: The dataset comprises `51290` entries, indicating a substantial volume of transactional data.
* **Columns**: There are `24` columns, each representing a specific attribute of the sales transactions.
* **Data Types**:
    * **Object Type**: `17` columns are of `object` type. Notably, `Order Date` and `Ship Date` are currently `object` type and will require conversion to `datetime` objects for proper time-series analysis.
    * **Numeric Types**: `5` columns are `float64` and `2` are `int64`, primarily representing quantitative measures like `Sales`, `Profit`, `Discount`, `Shipping Cost`, and `Quantity`.
* **Non-Null Counts**:
    * The `Postal Code` column shows a significant number of missing values (`9994` non-null out of `51290`), suggesting a potential lack of postal information for many entries.
    * All other critical analytical columns (e.g., `Sales`, `Profit`, `Quantity`, `Order ID`) are fully populated (`51290` non-null), which is ideal for subsequent analysis.

This initial inspection highlights the need for data type conversion and flags the `Postal Code` column for potential handling of missing values.

In [ ]:
df.head()

### 1.2 Snapshot of Raw Data from `df.head()`

Inspecting the first five rows of the DataFrame with `df.head()` provides a quick visual understanding of the data content and format:

* **Granularity**: Each row appears to represent a specific item within an order, showing details like `Product Name`, `Sales`, `Quantity`, `Discount`, and `Profit` per item.
* **Date Format**: The `Order Date` and `Ship Date` columns visually confirm their current string format (e.g., '31-07-2012'), reinforcing the necessity for `datetime` conversion.
* **Key Variables**: We can clearly see the presence of important categorical variables like `Ship Mode`, `Segment`, `Category`, and `Sub-Category`, which will be crucial for segmentation and categorization in our analysis and Power BI dashboard.
* **Numeric Values**: The `Sales`, `Profit`, `Discount`, and `Shipping Cost` columns display a variety of numerical values, giving an initial sense of the range and distribution of these financial metrics.

---

## 2. Data Cleaning and Preparation

In this section, we perform data cleaning to ensure the dataset is suitable for analysis and dashboarding.  
The main goals are:

- Converting date columns to proper datetime format  
- Optimizing categorical columns to reduce memory usage  
- Removing duplicated records  
- Reviewing and handling missing values (if applicable)

This will result in a clean and structured dataset ready for Power BI integration.

### 2.1 Converting Date Columns

We convert the `Order Date` and `Ship Date` columns from string to datetime format.  
The parameter `dayfirst=True` is used because the dataset follows the `DD-MM-YYYY` format.

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True)
df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True)

# Verify the conversion
df[['Order Date', 'Ship Date']].info()

### 2.2 Optimizing Categorical Columns

To improve memory efficiency and speed up future processing, we convert all object-type columns with **20 or fewer unique values** to the `category` data type.

In [ ]:
cols_cat = df.select_dtypes(include='object').nunique().sort_values()
cols_cat = cols_cat[cols_cat <= 20].index.tolist()

df[cols_cat] = df[cols_cat].astype('category')
df.info()

### 2.3 Removing Duplicated Rows

We identify and remove duplicated rows from the dataset to avoid data redundancy and inaccurate aggregations.

In [ ]:
print("Rows before removing duplicates:", df.shape[0])
print("Duplicated rows found:", df.duplicated().sum())
df.drop_duplicates(inplace=True)
print("Rows after removing duplicates:", df.shape[0])

### 2.4 Missing Values Check

To ensure data completeness and reliability for our analysis, we will check for missing values across all columns. Special attention will be given to the main numeric columns and KPIs: `Sales`, `Profit`, `Shipping Cost`, `Discount`, and `Quantity`.

In [ ]:
print(df[['Sales', 'Profit', 'Shipping Cost', 'Discount', 'Quantity']].isnull().sum())

### 2.5 Summary Statistics for Numerical Columns

We examine basic descriptive statistics for key numerical fields: `Sales`, `Profit`, `Shipping Cost`, `Discount`, and `Quantity`.

This provides a foundational understanding of value ranges and distributions before proceeding to outlier analysis.

In [ ]:
df[['Sales', 'Profit', 'Shipping Cost', 'Discount', 'Quantity']].describe()

#### 2.5.1 Interpretation of Numerical Statistics

The descriptive statistics provide critical initial insights into the distribution and range of our key numerical variables, highlighting important data characteristics:

* **Sales**: Exhibit a very wide range, from a minimum of `0.44` to a maximum of `22638.48`, with a mean of `246.49`. The absence of negative values for `Sales` suggests there are no erroneous entries or recorded returns in this column, indicating a clean transaction record. The high maximum value, significantly deviating from the mean, indicates the presence of exceptionally large sales, which may be **outliers** to be further investigated.
* **Profit**: Shows a substantial spread, ranging from a minimum of `-6599.98` (losses) to a maximum of `8399.98`. While the mean profit is `28.61`, the presence of negative values confirms transactions resulting in financial losses. Similar to Sales, the extremely high absolute values (both positive and negative) suggest the presence of **outliers** in Profit, warranting closer examination.
* **Shipping Cost**: Ranges from `0.0` to `933.57` with a mean of `26.38`. The minimum of `0.0` is plausible for certain shipping agreements. Like Sales, the absence of negative shipping costs suggests data integrity.
* **Discount**: Varies from `0.0` to `0.85` (85%), with an average of `0.14` (14%). The absence of negative discounts is expected, as discounts are reductions.
* **Quantity**: Shows a reasonable distribution from `1` to `14` units, with a mean of `3.48`. The minimum of `1` and maximum of `14` are within expected bounds for order quantities.

These statistics provide strong indicators of data quality (e.g., no negative sales/quantity) and identify potential areas of interest (e.g., high-value transactions and losses in `Sales` and `Profit`) that warrant a dedicated **outlier analysis**, as will be performed in the next steps.

### 2.6 Detecting Outliers (IQR Method)

We define a function to detect outliers using the Interquartile Range (IQR) method, which is a common technique to identify extreme values in continuous distributions.

Outliers are flagged in the following columns:

- `Sales`
- `Profit`
- `Shipping Cost`

In [ ]:
# Function to detect outliers using the IQR method
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

In [ ]:
# Identify outliers in Sales
outliers_sales = detect_outliers(df, 'Sales')
print(f'Outliers in Sales: {outliers_sales.shape[0]}')

# Identify outliers in Profit
outliers_profit = detect_outliers(df, 'Profit')
print(f'Outliers in Profit: {outliers_profit.shape[0]}')

# Identify outliers in Shipping Cost
outliers_shipping_cost = detect_outliers(df, 'Shipping Cost')
print(f'Outliers in Shipping Cost: {outliers_shipping_cost.shape[0]}')

### 2.7 Correlation Matrix

To understand potential linear relationships, we compute the Pearson correlation between the following variables:

- `Sales`
- `Profit`
- `Shipping Cost`
- `Discount`

This will help validate whether some outliers can be explained by expected interactions (e.g., high discounts reducing profit).

In [ ]:
correlation_matrix = df[['Sales', 'Profit','Shipping Cost', 'Discount']].corr()
print(correlation_matrix)

#### 2.7.1 Interpretation of the Correlation Matrix

The correlation matrix reveals several important relationships between the selected variables:
* **`Sales` and `Shipping Cost` (0.768)**: There is a strong positive correlation, indicating that transactions with higher sales volumes generally imply higher shipping costs.
* **`Sales` and `Profit` (0.485)**: A moderate positive correlation, suggesting that, in general, higher sales contribute to higher profits.
* **`Profit` and `Discount` (-0.316)**: A moderate negative correlation. This is a crucial point, as it confirms the hypothesis that applying discounts tends to negatively impact profit margins. This relationship is expected and helps contextualize some of the `Profit` outliers (losses) previously identified.
* The correlations between `Discount` and `Sales` (-0.087), and `Discount` and `Shipping Cost` (-0.079) are very weak, indicating that the discount alone does not have a strong direct linear relationship with sales volume or shipping costs in this data.

This correlation analysis provides a statistical understanding of the interactions between variables, which will be complemented by visual exploration.

### 2.8 Scatter Plots for Visual Exploration

We generate scatter plots to visually explore the impact of `Discount` on:

- `Sales`
- `Profit`

This step helps us understand whether the statistical outliers detected earlier are justified by real business dynamics.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Scatter plot of Sales vs Discount
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Discount', y='Sales', data=df)
plt.title('Sales vs Discount')
plt.show()

# Scatter plot of Profit vs Discount
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Discount', y='Profit', data=df)
plt.title('Profit vs Discount')
plt.show()

### 2.9 Why Outliers Were Not Removed

During initial exploration, an IQR-based method identified up to **21,319 records** (≈41% of the dataset) as potential outliers.  
The most extreme case was the `Profit` column, with **9,755 flagged entries**.

Such high volumes suggest that these values are **not necessarily errors**, but could reflect realistic, rare events — such as:

- Bulk orders
- High-margin or loss-leader products
- Logistics-driven pricing

To support this hypothesis, we analyzed the **correlation matrix** including `Discount`, revealing:

|                | Sales     | Profit    | Shipping Cost | Discount   |
|----------------|-----------|-----------|----------------|------------|
| **Sales**      | 1.000     | 0.485     | 0.768          | -0.087     |
| **Profit**     | 0.485     | 1.000     | 0.354          | -0.316     |
| **Shipping**   | 0.768     | 0.354     | 1.000          | -0.079     |
| **Discount**   | -0.087    | -0.316    | -0.079         | 1.000      |

We also visually inspected scatterplots and identified several data points that are distant from the norm. For example:

- At `Discount = 0.50`, most sales are below 5,000 — but one value exceeds **22,000**, possibly indicating a data entry error or a rare high-value sale.

However, because these cases do not distort the overall trends and may reflect **genuine market behavior**, we opted to:

**Acknowledge these as visual outliers**,  
**But keep them in the dataset** to preserve completeness and business realism.

In Power BI, such values can be flagged with visual cues or filters, allowing stakeholders to explore them as needed.

## 3. Exporting Clean Data for Power BI

After completing the data cleaning process, we export the dataset to a `.csv` file for use in Power BI.

The exported dataset includes:

- Cleaned and deduplicated sales records  
- Formatted datetime fields (`Order Date`, `Ship Date`)  
- Categorical columns optimized  
- No rows removed due to outliers — they were reviewed but retained for transparency

The exported file will serve as the data source for our dashboards.




In [ ]:
# Export clean data to CSV (ready for Power BI)
df.to_csv('/content/Global_Superstore_Clean.csv', index=False)

## 4. Power BI Dashboard Plan

The visualization layer of this project will be developed in **Power BI**, using the cleaned dataset.  
The dashboard will be structured into two thematic pages for clarity and impact.

### Page 1 – Sales Overview

This page will focus on general business performance and sales trends.

**Main visuals:**

- **KPI Cards**:  
  - Total Sales  
  - Total Profit  
  - Average Discount  

- **Line Chart**:  
  - Sales and Profit by Month

- **Bar Charts**:  
  - Sales by Segment  
  - Sales by Category  

- **Column Chart**:
  - Top 10 Sub-Categories by Sales

- **Filters (Slicers):**  
  - Order Year / Month  
  - Region
  - Segment
  - Category

### Page 2 – Customer & Order Behavior

This page will provide insights into order dynamics and customer behavior.

**Main visuals:**

- **KPI Cards:**  
  - Total Orders  
  - Total Distinct Customers  
  - Average Shipping Delay

- **Column Chart:**  
  - Quantity by Sub-Category

- **Scatter Plot:**  
  - Profit vs. Discount

- **Stacked Bar Chart:**  
  - Sales by Region and Category

- **Table (for outlier review):**  
  - Order ID
  - Customer Name
  - Sales
  - Discount
  - Profit
